<a href="https://colab.research.google.com/github/Qu1nnD/CS290/blob/main/Q%26CprojectNextVersion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.tree import plot_tree
import math
from sklearn.model_selection import train_test_split
from statistics import mean, stdev
from sklearn.metrics import accuracy_score

In [23]:
def Attribute_selection_method(dataset, target, method, classification):
  if(method == "classification"): #method chosen was classification
    features = dataset.columns[dataset.columns != target]
    entropies=[]
    for attribute in features:
      entropy = 0
      overall = len( dataset )
      vals = dataset[attribute].unique()
      if(classification =="entropy"): #classifcation was chosen and entropy was chosen
        for val in vals:
            subset= dataset[ dataset[attribute] == val ]
            subset_size = len(subset)
            weight = subset_size / overall
            if subset_size > 0:
              props = subset[target].value_counts( normalize=True )
              for p in props.array:
                entropy =  entropy - weight*(p*math.log2(p))
        entropies.append(entropy)
      elif classification == "Gini": #classification was chosen and Gini was chosen
          for val in vals:
            left = dataset[ dataset[attribute] <= val ]
            right = dataset[ dataset[attribute] > val ]

          for subset in [left, right]:
            subset_size = len(subset)
            weight = subset_size / overall
            if subset_size > 0:
              props = subset[target].value_counts( normalize=True )
              gini = 1 - sum(p**2 for p in props if p > 0)
              entropy += weight * gini

          entropies.append(entropy)

    minEnt=min(entropies)
    ind=entropies.index(minEnt)
    return features[ind]

  elif(method == "NaiveBayes"): # method is Naive Bayes Classifier/ regressor
    #separate by class
    test = [dataset[dataset!=target]]
    separated = {}
    for i in range(len(dataset)):
      vector = dataset.iloc[i].values
      class_value = vector[-1]
      if (class_value not in separated):
        separated[class_value] = []
      separated[class_value].append(vector)

    #Summarize statistic
    summaries = {}
    for class_value, instances in separated.items():
      numerical_instances = [[float(x) for x in instance[:-1] if isinstance(x, (int, float))] for instance in instances]
      if numerical_instances:
        summaries[class_value] = [(mean(col), stdev(col), len(col)) for col in zip(*numerical_instances)]
      else:
        summaries[class_value] = []


    predictions = []
    #put test into the method
    for index in range(len(test)):
        row = test.iloc[index].values.astype(float) #access row data and convert to float
        total_rows = sum(summaries[label][0][2] for label in summaries if summaries[label])
        probabilities = {}
        for class_value, class_summaries in summaries.items():
          if class_summaries:
            probabilities[class_value] = summaries[class_value][0][2]/float(total_rows)
            for i in range(len(class_summaries)):
              mean_val, stdev_val, _ = class_summaries[i]
              exponent = math.exp(-((row[i]-mean_val)**2 / (2 * stdev_val**2 )))
              probabilities[class_value] *=(1 / (math.sqrt(2 * math.pi) * stdev_val)) * exponent
        #determine the best label
        best_label, best_prob = None, -1
        for class_value, probability in probabilities.items():
          if best_label is None or probability > best_prob:
            best_prob = probability
            best_label = class_value

        predictions.append(best_label)
    return(predictions)

In [3]:
shrooms = pd.read_csv("https://raw.githubusercontent.com/Qu1nnD/CS290/main/data/mushroom_cleaned.csv") # categorical dataset
StudentPerform = pd.read_csv("https://github.com/Charlee0616/Data-Mining/raw/main/StudentPerformanceFactors.csv") # numerical dataset
shrooms.describe()
StudentPerform.head()


,Hours_Studied,Attendance,Parental_Involvement,Access_to_Resources,Extracurricular_Activities,Sleep_Hours,Previous_Scores,Motivation_Level,Internet_Access,Tutoring_Sessions,Family_Income,Teacher_Quality,School_Type,Peer_Influence,Physical_Activity,Learning_Disabilities,Parental_Education_Level,Distance_from_Home,Gender,Exam_Score
0,23,84,Low,High,No,7,73,Low,Yes,0,Low,Medium,Public,Positive,3,No,High School,Near,Male,67
1,19,64,Low,Medium,No,8,59,Low,Yes,2,Medium,Medium,Public,Negative,4,No,College,Moderate,Female,61
2,24,98,Medium,Medium,Yes,7,91,Medium,Yes,2,Medium,Medium,Public,Neutral,4,No,Postgraduate,Near,Male,74
3,29,89,Low,Medium,Yes,8,98,Medium,Yes,1,Medium,Medium,Public,Negative,4,No,High School,Moderate,Male,71
4,19,92,Medium,Medium,Yes,6,65,Medium,Yes,3,Medium,High,Public,Neutral,4,No,College,Near,Female,70


In [4]:
StudentPerform.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6607 entries, 0 to 6606
Data columns (total 20 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Hours_Studied               6607 non-null   int64 
 1   Attendance                  6607 non-null   int64 
 2   Parental_Involvement        6607 non-null   object
 3   Access_to_Resources         6607 non-null   object
 4   Extracurricular_Activities  6607 non-null   object
 5   Sleep_Hours                 6607 non-null   int64 
 6   Previous_Scores             6607 non-null   int64 
 7   Motivation_Level            6607 non-null   object
 8   Internet_Access             6607 non-null   object
 9   Tutoring_Sessions           6607 non-null   int64 
 10  Family_Income               6607 non-null   object
 11  Teacher_Quality             6529 non-null   object
 12  School_Type                 6607 non-null   object
 13  Peer_Influence              6607 non-null   obje

In [5]:
#Testing and Training data for Shrooms dataset
C_train_set, C_test_set = train_test_split(shrooms, test_size=0.2)
C_train_set = C_train_set[['cap-diameter', 'cap-shape', 'gill-attachment', 'gill-color', 'stem-height', 'stem-width', 'stem-color', 'class']]
C_test_set = C_test_set[['cap-diameter', 'cap-shape', 'gill-attachment', 'gill-color','stem-height', 'stem-width', 'stem-color', 'class']]
C_X_train = C_train_set[['cap-diameter', 'cap-shape', 'gill-attachment', 'gill-color','stem-height', 'stem-width', 'stem-color']]
C_y_train = C_train_set["class"]
C_X_test = C_test_set[['cap-diameter', 'cap-shape', 'gill-attachment', 'gill-color','stem-height', 'stem-width', 'stem-color']]
C_y_test = C_test_set["class"]
cat_attributes=[]
num_attributes=[]
for col in C_X_train.columns:
  if len(C_X_train[col].unique()) < 20:
    cat_attributes.append(col)
  else:
    num_attributes.append(col)
trf = [ ('num', StandardScaler(), num_attributes),
       ('cat', OneHotEncoder( handle_unknown='ignore'), cat_attributes) ]
col_transform = ColumnTransformer( transformers = trf )
pipeline = Pipeline( steps = [('pre', col_transform),
 ('clf', DecisionTreeClassifier())])

In [6]:
#Training the data from the Student performance dataset
R_train_set, R_test_set = train_test_split(StudentPerform, test_size=0.2)
num_attributes = R_train_set.select_dtypes( include = ['int64','float64']).columns
R_X_train = R_train_set[num_attributes[num_attributes!="Exam_Score"]]
R_y_train = R_train_set["Exam_Score"]
R_X_test = R_test_set[num_attributes[num_attributes!="Exam_Score"]]
R_y_test = R_test_set["Exam_Score"]

In [7]:
CatentropyTest = Attribute_selection_method(shrooms, "class", "classification", "entropy")
CatentropyTest

'stem-width'

In [8]:
NumentropyTest = Attribute_selection_method(StudentPerform, "Exam_Score", "classification", "entropy")
NumentropyTest

'Attendance'

In [9]:
CatginiTest = Attribute_selection_method(shrooms, "class", "classification", "Gini")
CatginiTest

'cap-shape'

In [150]:
NumRegressorTest = Attribute_selection_method(StudentPerform, "Exam_Score", "NaiveBayes", "regression")
NumRegressorTest

AttributeError: 'list' object has no attribute 'iloc'

In [155]:
dataset=StudentPerform
target="Exam_Score"
method="NaiveBayes"
num_attributes = dataset.select_dtypes( include = ['int64','float64']).columns
test = dataset[num_attributes[num_attributes!=target]]
#Summarize statistic
summaries = []
print(test)
for i in range(len(test.columns)):
  meany=[]
  for j in range(len(test)):
    meany.append(test[name].get(j))
  name=test.columns[i]
  data=([name],(mean(meany)),(stdev(meany)))
  summaries.append(data)
predictions = []
#put test into the method
for index in range(len(dataset.columns)):
  total_rows = len(test)
  probabilities = []
  class_summaries=len(summaries)
  for i in range(class_summaries):
    probabilities.append(summaries[0][2]/float(total_rows))
    stdev_val=summaries[i][2]
    mean_val=summaries[i][1]
    exponent = math.exp(-((row{1}-mean_val)**2 / (2 * stdev_val**2 ))) #Fix the row{1} thing later
    probabilities[i] *=(1 / (math.sqrt(2 * math.pi) * stdev_val)) * exponent
  #determine the best label
  best_label, best_prob = None, -1
  probability=10 # Fix for later
  for k in range(len(probabilities)):
    if best_label is None or probability > best_prob:
      best_prob = probabilities[k]
      best_label = summaries[k][0]
  predictions.append(best_label)
print(probabilities)
predictions

      Hours_Studied  Attendance  Sleep_Hours  Previous_Scores  \
0                23          84            7               73   
1                19          64            8               59   
2                24          98            7               91   
3                29          89            8               98   
4                19          92            6               65   
...             ...         ...          ...              ...   
6602             25          69            7               76   
6603             23          76            8               81   
6604             20          90            6               65   
6605             10          86            6               91   
6606             15          67            9               94   

      Tutoring_Sessions  Physical_Activity  
0                     0                  3  
1                     2                  4  
2                     2                  4  
3                     1                

[['Physical_Activity'],
 ['Physical_Activity'],
 ['Physical_Activity'],
 ['Physical_Activity'],
 ['Physical_Activity'],
 ['Physical_Activity'],
 ['Physical_Activity'],
 ['Physical_Activity'],
 ['Physical_Activity'],
 ['Physical_Activity'],
 ['Physical_Activity'],
 ['Physical_Activity'],
 ['Physical_Activity'],
 ['Physical_Activity'],
 ['Physical_Activity'],
 ['Physical_Activity'],
 ['Physical_Activity'],
 ['Physical_Activity'],
 ['Physical_Activity'],
 ['Physical_Activity']]